In [5]:
import torch
import glob
from PIL import Image
import torch.optim as optim
from torch.utils.data import DataLoader,  TensorDataset, Dataset
import torch.nn as nn
import torchvision
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
import math

from os import listdir
import shutil
import time
from tqdm import tqdm

import imageio
import time

import pickle

from google.colab import drive
drive.mount('/gdrive')

import random
random.seed(42)

Mounted at /gdrive


In [ ]:
# dataset creating 
class MyDataset(Dataset):
    
    def __init__(self, data, targets, transform=None):
        self.dir_data = data
        self.dir_targets = targets

        self.list_data = listdir(data)
        self.list_targets = listdir(targets)

        self.list_data.sort()
        self.list_targets.sort()
 
    def __len__(self):
        if len(listdir(self.dir_data)) != len(listdir(self.dir_targets)):
            raise Exception(f'error {len(listdir(self.dir_data))} {len(listdir(self.dir_targets))}')
        return len(listdir(self.dir_data))

    def __getitem__(self, idx):
        with open(f'{self.dir_data}/{self.list_data[idx]}', 'rb') as f:
            image = pickle.load(f)

        with open(f'{self.dir_targets}/{self.list_targets[idx]}', 'rb') as f:
            label = pickle.load(f)

        return torch.from_numpy(image).float(), torch.from_numpy(label)

In [ ]:
# dataset
dir_img = '/gdrive/My Drive/TyurinaAV_segmentation/DatasetCreating/datasets/train_val_images/'
dir_mask = '/gdrive/My Drive/TyurinaAV_segmentation/DatasetCreating/datasets/train_val_masks/'

dataset = MyDataset(data=dir_img, targets=dir_mask)

In [ ]:
# classes balancing
# indices of images with total black and wb zones
counter_black = []
counter_wb = []
for i in tqdm(range(len(dataset))):
    if i%50 == 0:
        time.sleep(10)
    if dataset[i][1].max() > 0:
        counter_wb.append(i)
        #print(i,'wb')
    else:
        counter_black.append(i)
        #print(i,'b')

print(len(counter_black))
print(len(counter_wb))
#print(len(set(counter_wb))+len(set(counter_black)))

In [ ]:
# chosing random images of the same amount
ind_black = random.sample(counter_black, 2600)
ind_wb = random.sample(counter_wb, 2600)
#len(set(ind_black))+len(set(ind_wb))

In [ ]:
ind = ind_black + ind_wb
print(len(set(ind)))

In [ ]:
# copying files from full dataset to creat balanced dataset
for index in tqdm((ind)):
    #if i%100 == 0:
    #    time.sleep(10)
    shutil.copy(f'/gdrive/My Drive/TyurinaAV_segmentation/DatasetCreating/datasets/train_val_images/image_{index}', f'/gdrive/My Drive/TyurinaAV_segmentation/DatasetCreating/datasets/train_val_images_balanced/image_{index}')
    shutil.copy(f'/gdrive/My Drive/TyurinaAV_segmentation/DatasetCreating/datasets/train_val_masks/mask_{index}', f'/gdrive/My Drive/TyurinaAV_segmentation/DatasetCreating/datasets/train_val_masks_balanced/mask_{index}')